In [1]:
from HATN import *
import numpy as np
import pandas as pd
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

Using TensorFlow backend.


In [2]:
with open(os.path.join(os.getcwd(), '../Data/trainingandtestdata/training.1600000.processed.noemoticon.csv'), 
          'r', encoding='latin-1') as f:
    data = pd.read_csv(f, names=["Polarity", "ID", "Date", "Query", "User", "Text"])

In [9]:
data.head()

,Polarity,Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [3]:
data.drop(labels=['Query', 'Date', 'User', 'ID'], axis=1, inplace=True)

In [4]:
# Ham: 0, Spam: 1
labels = data['Polarity']

In [5]:
weight_path = os.path.join(os.getcwd(), '../Data/glove.840B.300d.txt')

In [6]:
(x_train, y_train), (x_val, y_val), word_index, embedding_matrix = text_processing(data['Text'].values, labels, weights_path=weight_path)

Total 690960 unique tokens.
Shape of data tensor: (1600000, 8, 15)
Shape of label tensor: (1600000,)
Number of spams in the training set is 2560936/1280000
Number of spams in the training set is 639064/320000
Total 2195885 word vectors.


In [7]:
model = HATN(len(word_index) + 1, numclass=5, trainable=False, embedding_matrix = embedding_matrix)

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [9]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

print(y_train.shape)

(1280000, 5)


In [20]:
import time
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, LearningRateScheduler, TensorBoard, RemoteMonitor
# Initialize Keras Callbacks
checkpoints = ModelCheckpoint(filepath='metadata/weights.epoch{epoch:02d}-val_loss{val_loss:.2f}.hdf5', 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=False, 
                             save_weights_only=True, 
                             mode='min', 
                             period=1)

early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=0.0,
                               patience=15,
                               verbose=1)

csv_logger = CSVLogger(filename='training_log.csv',
                       separator=',',
                       append=True)

remote = RemoteMonitor()

tensorboard = TensorBoard(log_dir="logs/{}".format(time.time()), 
                          histogram_freq=0, 
                          batch_size=64, 
                          write_graph=True, 
                          write_grads=True, 
                          write_images=True, 
                          update_freq='batch')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                                         factor=0.2,
                                         patience=15, 
                                         min_lr=0.001)

callbacks = [checkpoints, 
            early_stopping, 
            csv_logger,
            tensorboard, 
            reduce_lr, 
            remote]


In [19]:
import keras
model2 = keras.models.load_model(os.path.join(os.getcwd(), 'metadata/weights.epoch01-val_loss0.40.hdf5'), custom_objects={'AttLayer': AttLayer})

TypeError: __init__() got an unexpected keyword argument 'name'

In [21]:
batch_size = 64
total_epochs = 50
initial_epoch = 0
validation_split = 0.2

print("model fitting - Hierachical attention network")

history = model.fit(x=x_train, 
                    y=y_train, 
                    batch_size=batch_size,
                    epochs=total_epochs,
                    verbose=1, 
                    callbacks=callbacks,
                    validation_split=validation_split, 
                    shuffle=True,
                    initial_epoch=initial_epoch)

model fitting - Hierachical attention network
Train on 1024000 samples, validate on 256000 samples
Epoch 1/50
1024000/1024000 [==============================] - 4869s 5ms/step - loss: 0.3792 - acc: 0.8287 - val_loss: 0.3891 - val_acc: 0.8249

Epoch 00001: saving model to metadata/weights.epoch01-val_loss0.39.hdf5
Epoch 2/50
1024000/1024000 [==============================] - 5807s 6ms/step - loss: 0.3613 - acc: 0.8385 - val_loss: 0.3853 - val_acc: 0.8263

Epoch 00002: saving model to metadata/weights.epoch02-val_loss0.39.hdf5
Epoch 3/50
1024000/1024000 [==============================] - 5548s 5ms/step - loss: 0.3467 - acc: 0.8459 - val_loss: 0.3913 - val_acc: 0.8241

Epoch 00003: saving model to metadata/weights.epoch03-val_loss0.39.hdf5
Epoch 4/50
1024000/1024000 [==============================] - 5209s 5ms/step - loss: 0.3340 - acc: 0.8528 - val_loss: 0.4003 - val_acc: 0.8225

Epoch 00004: saving model to metadata/weights.epoch04-val_loss0.40.hdf5
Epoch 5/50
 296128/1024000 [=======>.

KeyboardInterrupt: 